In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


Get Jobs Numbers from the metros

In [29]:
class DeptOfNumbersData:
    def __init__(self):
        self.dept_numbers_path = "https://www.deptofnumbers.com/"
        self.dn_jobs_path = "employment/metros/"
        self.get_soup()
        self.get_month()
        
    def get_soup(self):       
        page = requests.get(self.dept_numbers_path + self.dn_jobs_path)
        soup = BeautifulSoup(page.text, 'html.parser')
        self.soup = soup
        
    def get_month(self):
        table = self.soup.find('table', {'id': 'metro_table'})
        table_rows = table.find_all('th')
        self.month = table_rows[1].text[:-16]

    def get_jobs_numbers(self, write_csv = False):
        table = self.soup.find('table', {'id': 'metro_table'})
        table_rows = table.find_all('tr')

        l = self.get_table(table_rows)
        
        df = pd.DataFrame(l,columns=["Metro","Month Growth", "Month Growth (%)", "Year Growth", "Year Growth (%)"]).drop(0)
        df = df.sort_values(by="Year Growth (%)", ascending=False)
        
        if (write_csv):
            test = self.month + ".csv"
            df.to_csv("JobsData/" + test)
            
        self.jobs_numbers = df
        return df

    
    @staticmethod
    def get_table(table_rows):
        l = []
        for tr in table_rows:
            td = tr.find_all('td')
            row = [tr.text.strip(" %") for tr in td if tr.text.strip()]
            l.append(row)
        return l
    
    def find_city(self, city_name):
        cityName = city_name.lower()
        metroNames = self.jobs_numbers["Metro"].str.lower()
        return self.jobs_numbers[metroNames.str.contains(city_name)]

In [30]:
dept = DeptOfNumbersData()
dept.get_soup()

df = dept.get_jobs_numbers(write_csv=True)
df[df["Year Growth (%)"].astype(float) > 3]

,Metro,Month Growth,Month Growth (%),Year Growth,Year Growth (%)
317,"Yuba City, CA",200,0.41,"3,800",8.39
62,"Cleveland, TN",200,0.40,"2,500",5.27
282,"State College, PA","3,000",3.73,"3,100",3.86
279,"St George, UT",300,0.42,"2,500",3.59
236,"Provo, UT",200,0.07,"9,300",3.48
21,"Austin, TX","1,400",0.12,"38,100",3.47
256,"Salt Lake City, UT","1,700",0.22,"25,600",3.45
65,"College Station, TX",800,0.64,"4,100",3.39
129,"Hinesville, GA",200,0.93,700,3.32
228,"Phoenix, AZ","1,500",0.07,"70,600",3.29


'January 2020'

In [5]:
dept.find_city("tampa")

,Metro,Month Growth,Month Growth (%),Year Growth,Year Growth (%)
287,"Tampa, FL","9,800",0.70,"40,600",2.96


In [ ]:
api.census.gov/data/2018/acs/acs1?get=NAME,group(B01001)&for=us:1&key=YOUR_KEY_GOES_HERE